In [3]:
!pip install ortools


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.9/24.9 MB 96.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 15.0 MB/s eta 0:00:00
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0


In [5]:
#Task 1
"""
a) CSP Definition
Variables: Each step position (x, y) the robot can take in the grid.

Domains: All cells in the grid excluding obstacles, and movement restricted to diagonal neighbors.

Constraints:

Robot can only move diagonally (i.e., to 4 diagonal neighbors)

Must avoid obstacle cells

Must start at S=(1,1) and end at T=(4,4)

Path must be connected and valid
"""

from ortools.sat.python import cp_model
import math

grid_size = 5
start = (1, 1)
goal = (4, 4)
obstacles = []

diagonals = [(-1, -1), (-1, 1), (1, -1), (1, 1)]

class PathPrinter(cp_model.CpSolverSolutionCallback):
    def __init__(self, x_vars, y_vars, steps):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.x_vars = x_vars
        self.y_vars = y_vars
        self.steps = steps
        self.best_path = []

    def on_solution_callback(self):
        self.best_path = [(self.Value(self.x_vars[i]), self.Value(self.y_vars[i])) for i in range(self.steps)]

def solve_diagonal_path(grid_size, start, goal, obstacles):
    model = cp_model.CpModel()
    max_steps = grid_size * 2
    x = [model.NewIntVar(0, grid_size - 1, f'x{i}') for i in range(max_steps)]
    y = [model.NewIntVar(0, grid_size - 1, f'y{i}') for i in range(max_steps)]

    model.Add(x[0] == start[0])
    model.Add(y[0] == start[1])
    model.Add(x[-1] == goal[0])
    model.Add(y[-1] == goal[1])

    for i in range(max_steps - 1):
        dx = model.NewIntVarFromDomain(cp_model.Domain.FromValues([-1, 1]), f'dx{i}')
        dy = model.NewIntVarFromDomain(cp_model.Domain.FromValues([-1, 1]), f'dy{i}')
        model.Add(x[i + 1] == x[i] + dx)
        model.Add(y[i + 1] == y[i] + dy)

        for obs in obstacles:
            model.AddForbiddenAssignments([x[i + 1], y[i + 1]], [obs])

    for i in range(max_steps):
        for j in range(i + 1, max_steps):
            bx = model.NewBoolVar(f'bx_{i}_{j}')
            by = model.NewBoolVar(f'by_{i}_{j}')
            model.Add(x[i] != x[j]).OnlyEnforceIf(bx)
            model.Add(x[i] == x[j]).OnlyEnforceIf(bx.Not())
            model.Add(y[i] != y[j]).OnlyEnforceIf(by)
            model.Add(y[i] == y[j]).OnlyEnforceIf(by.Not())
            model.AddBoolOr([bx, by])

    solver = cp_model.CpSolver()
    printer = PathPrinter(x, y, max_steps)
    solver.parameters.max_time_in_seconds = 5.0
    solver.Solve(model, printer)

    path = [p for p in printer.best_path if p[0] >= 0 and p[0] < grid_size and p[1] >= 0 and p[1] < grid_size]
    seen = set()
    final_path = []
    for p in path:
        if p not in seen:
            seen.add(p)
            final_path.append(p)
        if p == goal:
            break

    distance = (len(final_path) - 1) * math.sqrt(2)
    return final_path, distance

path, dist = solve_diagonal_path(5, (1, 1), (4, 4), [])
print("Shortest path:", path)
print("Total distance:", dist)


Shortest path: []
Total distance: -1.4142135623730951


In [16]:
#Task 2

from ortools.sat.python import cp_model

def compute_perimeter(grid):
    rows = len(grid)
    if rows == 0:
        return 0
    cols = len(grid[0])

    model = cp_model.CpModel()


    in_landmass = [[model.NewBoolVar(f'in_landmass_{r}_{c}') for c in range(cols)] for r in range(rows)]


    perimeter_contrib = [[model.NewIntVar(0, 4, f'perim_{r}_{c}') for c in range(cols)] for r in range(rows)]


    for r in range(rows):
        for c in range(cols):
            if grid[r][c] == 0:
                model.Add(in_landmass[r][c] == 0)

    for r in range(rows):
        for c in range(cols):

            count = 0
            if r == 0 or grid[r-1][c] == 0:
                count += 1
            if r == rows-1 or grid[r+1][c] == 0:
                count += 1
            if c == 0 or grid[r][c-1] == 0:
                count += 1
            if c == cols-1 or grid[r][c+1] == 0:
                count += 1
            model.Add(perimeter_contrib[r][c] == count).OnlyEnforceIf(in_landmass[r][c])
            model.Add(perimeter_contrib[r][c] == 0).OnlyEnforceIf(in_landmass[r][c].Not())


    landmass_size = model.NewIntVar(0, rows*cols, 'landmass_size')
    model.Add(landmass_size == sum(in_landmass[r][c] for r in range(rows) for c in range(cols)))
    model.Maximize(landmass_size)


    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    if status == cp_model.OPTIMAL:

        total_perimeter = 0
        for r in range(rows):
            for c in range(cols):
                if solver.Value(in_landmass[r][c]):
                    total_perimeter += solver.Value(perimeter_contrib[r][c])
        return total_perimeter
    return 0

def main():
    island = [
        [0, 0, 0, 0, 0],
        [0, 1, 1, 0, 0],
        [0, 1, 1, 1, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0]
    ]
    perimeter = compute_perimeter(island)
    print(f"Perimeter of largest landmass: {perimeter}")

if __name__ == "__main__":
    main()

Perimeter of largest landmass: 12


In [23]:
#Task 3
from ortools.sat.python import cp_model
import math


cities = [
    (0, 0), (1, 3), (3, 1), (5, 0), (7, 3),
    (8, 8), (6, 7), (4, 6), (2, 8), (0, 5)
]

def distance(city1, city2):
    return math.sqrt((city2[0] - city1[0]) ** 2 + (city2[1] - city1[1]) ** 2)


n = len(cities)
dist_matrix = [[distance(cities[i], cities[j]) for j in range(n)] for i in range(n)]


def solve_tsp():
    model = cp_model.CpModel()


    x = [[model.NewBoolVar(f'x_{i}_{j}') for j in range(n)] for i in range(n)]

    u = [model.NewIntVar(0, n-1, f'u_{i}') for i in range(n)]


    for i in range(n):
        model.Add(sum(x[i][j] for j in range(n) if i != j) == 1)
        model.Add(sum(x[j][i] for j in range(n) if i != j) == 1)


    for i in range(1, n):
        for j in range(1, n):
            if i != j:
                model.Add(u[i] - u[j] + 1 <= (n - 1) * (1 - x[i][j]))


    model.Minimize(
        sum(dist_matrix[i][j] * x[i][j] for i in range(n) for j in range(n) if i != j)
    )


    solver = cp_model.CpSolver()
    solver.parameters.max_time_in_seconds = 10.0
    status = solver.Solve(model)

    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:

        tour = []
        for i in range(n):
            for j in range(n):
                if i != j and solver.Value(x[i][j]):
                    tour.append((i, j))
        return tour, solver.ObjectiveValue()
    else:
        return None, None


tour, total_distance = solve_tsp()

if tour:
    print("Optimal tour:", tour)
    print("Total distance:", total_distance)
else:
    print("No solution found.")


Optimal tour: [(0, 1), (1, 9), (2, 0), (3, 2), (4, 3), (5, 4), (6, 5), (7, 6), (8, 7), (9, 8)]
Total distance: 30.407376419602873


In [28]:
#Task 4

"""
1. CSP Model Definition (Variables, Domains, and Constraints)
a) Variables:
Robot positions: Each robot has a variable representing its current position in the grid (e.g., (x, y)).

Package assignments: A variable for each package that assigns the package to one robot.

Battery levels: A variable for each robot to track its battery level at different time steps.

Robot route: For each robot, a series of variables representing the path it will take from the package pickup location to the delivery location.

b) Domains:
Robot positions: The domain for each robot’s position is all possible locations in the 6x6 grid.

Package assignments: The domain for each package is the set of robots that have enough capacity to carry the package.

Battery levels: The battery level for each robot is a range from 0 to the maximum battery capacity, which decreases as the robot moves or makes a delivery.

c) Constraints:
Collision Avoidance: Robots cannot occupy the same grid position at the same time.

Carrying Capacity: A robot can only carry packages within its weight limit.

Pickup and Delivery: Each package must be picked up by a robot and delivered to its designated delivery point.

Battery Level Management: Robots must return to charging stations if their battery level is low.

Minimize Total Delivery Time: The goal is to minimize the total time spent by all robots completing their deliveries.
"""

from ortools.sat.python import cp_model

def warehouse_robot_csp():
    model = cp_model.CpModel()

    grid_size = 6
    num_robots = 5
    num_packages = 10
    max_steps = 15

    x = {}
    y = {}

    for r in range(num_robots):
        for t in range(max_steps):
            x[r, t] = model.NewIntVar(0, grid_size - 1, f"x_{r}_{t}")
            y[r, t] = model.NewIntVar(0, grid_size - 1, f"y_{r}_{t}")

    package_status = {}
    for p in range(num_packages):
        package_status[p] = model.NewIntVar(0, 2, f"package_{p}")

    for r in range(num_robots):
        for t in range(1, max_steps):
            dx = model.NewIntVar(-1, 1, f"dx_{r}_{t}")
            dy = model.NewIntVar(-1, 1, f"dy_{r}_{t}")

            model.Add(dx == x[r, t] - x[r, t - 1])
            model.Add(dy == y[r, t] - y[r, t - 1])

            model.AddAbsEquality(model.NewIntVar(0, 1, f"abs_dx_{r}_{t}"), dx)
            model.AddAbsEquality(model.NewIntVar(0, 1, f"abs_dy_{r}_{t}"), dy)

    for t in range(max_steps):
        for r1 in range(num_robots):
            for r2 in range(r1 + 1, num_robots):
                not_same_x = model.NewBoolVar(f"not_same_x_{r1}_{r2}_{t}")
                not_same_y = model.NewBoolVar(f"not_same_y_{r1}_{r2}_{t}")

                model.Add(x[r1, t] != x[r2, t]).OnlyEnforceIf(not_same_x)
                model.Add(y[r1, t] != y[r2, t]).OnlyEnforceIf(not_same_y)

                model.AddBoolOr([not_same_x, not_same_y])

    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    if status in [cp_model.OPTIMAL, cp_model.FEASIBLE]:
        print("\nOptimal Delivery Plan:")
        for t in range(max_steps):
            for r in range(num_robots):
                print(f"Robot {r} at step {t}: ({solver.Value(x[r, t])}, {solver.Value(y[r, t])})")
    else:
        print("No feasible solution found.")


warehouse_robot_csp()



Optimal Delivery Plan:
Robot 0 at step 0: (1, 0)
Robot 1 at step 0: (2, 3)
Robot 2 at step 0: (0, 1)
Robot 3 at step 0: (0, 0)
Robot 4 at step 0: (2, 2)
Robot 0 at step 1: (0, 1)
Robot 1 at step 1: (1, 2)
Robot 2 at step 1: (1, 0)
Robot 3 at step 1: (1, 1)
Robot 4 at step 1: (2, 3)
Robot 0 at step 2: (0, 2)
Robot 1 at step 2: (0, 1)
Robot 2 at step 2: (1, 1)
Robot 3 at step 2: (2, 0)
Robot 4 at step 2: (1, 3)
Robot 0 at step 3: (0, 1)
Robot 1 at step 3: (1, 1)
Robot 2 at step 3: (0, 2)
Robot 3 at step 3: (1, 0)
Robot 4 at step 3: (2, 3)
Robot 0 at step 4: (1, 0)
Robot 1 at step 4: (0, 1)
Robot 2 at step 4: (0, 2)
Robot 3 at step 4: (0, 0)
Robot 4 at step 4: (2, 2)
Robot 0 at step 5: (2, 0)
Robot 1 at step 5: (0, 0)
Robot 2 at step 5: (0, 2)
Robot 3 at step 5: (1, 1)
Robot 4 at step 5: (2, 1)
Robot 0 at step 6: (2, 1)
Robot 1 at step 6: (0, 1)
Robot 2 at step 6: (1, 1)
Robot 3 at step 6: (2, 2)
Robot 4 at step 6: (1, 0)
Robot 0 at step 7: (1, 0)
Robot 1 at step 7: (0, 0)
Robot 2 at ste

In [31]:
#Task 5
"""
a) Formulating the CSP
We need to represent the Sudoku puzzle as a constraint satisfaction problem (CSP). The constraints we have to consider are:

Sudoku Constraints:

Each row must contain unique numbers from 1 to 9.

Each column must contain unique numbers from 1 to 9.

Each 3×3 subgrid must contain unique numbers from 1 to 9.

Additional Constraints:

The sum of numbers in each diagonal must be divisible by 3.

Prime numbers (2, 3, 5, 7) cannot be adjacent (horizontally or vertically).

Variables: Each cell in the Sudoku grid will be a variable. This will be a 9x9 grid, meaning there are 81 variables, one for each cell.

Domains: The domain of each variable is the set of integers from 1 to 9, as these are the only valid entries for a standard Sudoku puzzle.

Constraints:

Row, Column, and Subgrid Uniqueness: For each row, column, and 3x3 subgrid, all numbers must be distinct.

Diagonal Sum Divisible by 3: The sum of the numbers in both diagonals (from top-left to bottom-right and from top-right to bottom-left) must be divisible by 3.

Prime Number Non-adjacency: Prime numbers (2, 3, 5, 7) must not be adjacent (either horizontally or vertically).
"""




from ortools.sat.python import cp_model

def solve_sudoku(grid):
    model = cp_model.CpModel()

    SIZE = 9
    SUBGRID = 3

    cells = [[model.NewIntVar(1, 9, f'cell_{r}_{c}') for c in range(SIZE)] for r in range(SIZE)]

    for r in range(SIZE):
        for c in range(SIZE):
            if grid[r][c] != 0:
                model.Add(cells[r][c] == grid[r][c])

    for r in range(SIZE):
        model.AddAllDifferent(cells[r])

    for c in range(SIZE):
        model.AddAllDifferent([cells[r][c] for r in range(SIZE)])

    for r in range(0, SIZE, SUBGRID):
        for c in range(0, SIZE, SUBGRID):
            subgrid_cells = [cells[r + dr][c + dc] for dr in range(SUBGRID) for dc in range(SUBGRID)]
            model.AddAllDifferent(subgrid_cells)

    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    if status == cp_model.FEASIBLE or status == cp_model.OPTIMAL:
        solved_grid = [[solver.Value(cells[r][c]) for c in range(SIZE)] for r in range(SIZE)]
        for row in solved_grid:
            print(row)
    else:
        print("No solution found.")

grid = [
    [5, 3, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9]
]

solve_sudoku(grid)


[5, 3, 4, 6, 7, 8, 9, 1, 2]
[6, 7, 2, 1, 9, 5, 3, 4, 8]
[1, 9, 8, 3, 4, 2, 5, 6, 7]
[8, 5, 9, 7, 6, 1, 4, 2, 3]
[4, 2, 6, 8, 5, 3, 7, 9, 1]
[7, 1, 3, 9, 2, 4, 8, 5, 6]
[9, 6, 1, 5, 3, 7, 2, 8, 4]
[2, 8, 7, 4, 1, 9, 6, 3, 5]
[3, 4, 5, 2, 8, 6, 1, 7, 9]
